In [1]:
import warnings

# 阻止FutureWarning输出
warnings.simplefilter(action='ignore', category=FutureWarning)

# 2 利用funct.Array实现链式计算

## 2.1 funct.Array的创建

In [2]:
from funct import Array

- **从其他数据结构创建**

In [3]:
# 从多个单元素创建
array1 = Array(1, 2, 3)
array1

Array(1, 2, 3)

In [4]:
# 从列表创建
array2 = Array([1, 2, 3])
array2

Array(1, 2, 3)

In [5]:
import numpy as np

# 从numpy数组创建
array3 = Array(np.array(range(1, 4)))
array3

Array(1, 2, 3)

- **类似numpy风格的规则创建方法**

In [6]:
# 创建0到10之间步长为2的序列
Array.arange(0, 10, 2)

Array(0, 2, 4, 6, 8)

In [7]:
# 在0到10之间等距取11个值生成序列
Array.linspace(0, 10, 11)

Array(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0)

- **创建嵌套Array**

In [8]:
Array([[0, 1], [2, 3], [4, 5, 6]])

Array(Array(0, 1), Array(2, 3), Array(4, 5, 6))

In [9]:
Array([1, 2, 3], [4, 5])

Array(Array(1, 2, 3), Array(4, 5))

In [10]:
Array(range(3), range(3))

Array(Array(0, 1, 2), Array(0, 1, 2))

In [11]:
Array(np.array(range(3)), np.array(range(3)))

Array(array([0, 1, 2]), array([0, 1, 2]))

In [12]:
Array(np.array(range(3)), np.array(range(3))).toArray()

Array(Array(0, 1, 2), Array(0, 1, 2))

## 2.2 funct.Array的索引

- **列表式索引**

In [13]:
array1 = Array(range(10))

array1[0]

0

In [14]:
array1[-1]

9

In [15]:
array1[2:8]

Array(2, 3, 4, 5, 6, 7)

In [16]:
array1[2:8:2]

Array(2, 4, 6)

- **数组式索引**

In [17]:
array1[[2, 3, 5, 7]]

Array(2, 3, 5, 7)

- **Bool值索引**

In [18]:
array1 >= 5

Array(False, False, False, False, False, True, True, True, True, True)

In [19]:
array1[array1 >= 5]

Array(5, 6, 7, 8, 9)

- **多层索引**

In [20]:
array2 = Array(range(3), range(3, 6), [range(6, 9), range(9, 12)])
array2

Array(Array(0, 1, 2), Array(3, 4, 5), Array(Array(6, 7, 8), Array(9, 10, 11)))

In [21]:
array2[:, 0]

Array(0, 3, Array(6, 7, 8))

In [22]:
array2[:-1, 2]

Array(2, 5)

In [23]:
# 因为Array支持不规则嵌套，所以索引不到的时候会报错
array2[:, 3]

IndexError: list index out of range

## 2.3 funct.Array的链式骚操作

- **level1：常规的数值运算**

In [24]:
array1 = Array(range(10))

(
    array1
    # 给每个位置加上2
    .add(2)
    # 给每个位置分别乘以不同的值
    .mul(range(10))
    # 每个位置除以3
    .div(3)
    # 求均值
    .mean()
)

12.5

- **level2：配合map方法推广元素级别运算**

In [25]:
# 标准化
(
    array1
    # 减去均值
    .add(-array1.mean())
    # 除以标准差
    .div(np.std(array1))
    # 保留4位小数
    .round(4)
)

Array(-1.5667, -1.2185, -0.8704, -0.5222, -0.1741, 0.1741, 0.5222, 0.8704, 1.2185, 1.5667)

In [26]:
# 归一化
(
    array1
    # 减去最小值
    .add(-array1.min())
    # 除以极差
    .div(array1.max() - array1.min())
    # 保留4位小数
    .round(4)
)

Array(0.0, 0.1111, 0.2222, 0.3333, 0.4444, 0.5556, 0.6667, 0.7778, 0.8889, 1.0)

- **level2：配合map方法推广元素级别运算**

In [27]:
(
    array1
    # 减去最小值
    .add(-array1.min())
    # 除以极差
    .div(array1.max() - array1.min())
    # 保留4位小数
    .round(4)
    # 对归一化结果分箱
    .map(lambda x: -1 if x <= 0.5 else 1)
)

Array(-1, -1, -1, -1, -1, 1, 1, 1, 1, 1)

- **level3：配合zip方法引入其他Array参与运算**

In [28]:
array1 = Array(np.random.rand(5))
array2 = Array(np.random.rand(5))
array3 = Array(np.random.rand(5))

(
    array1
    # 将array2与array3加入运算
    .zip(array2, array3)
    # 计算每个Array相同位置最大值
    .map(max)
)

Array(0.8511926766400677, 0.8271204035837049, 0.8095105055504694, 0.9914082852095631, 0.912115952796735)

- **level4：条件分组**

In [29]:
array1 = Array(np.random.rand(100))

(
    array1
    # 以0.33、0.66作为临界点对数据进行分组
    .groupBy(lambda x: 
             0 if x <= 0.33
             else (1 if x <= 0.66 else 2)
            )
    # 分组计数，这里先取出嵌套分组结果中包含源数据的部分
    [:, 1]
    .map(len)
)

Array(36, 31, 33)